In [1]:
import numpy as np 
import pandas as pd 
import cv2
import time
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import pickle
import os
from keras.models import load_model
from IPython.display import FileLink

2023-01-09 09:12:43.088652: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-09 09:12:43.281477: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/teresababio/miniconda3/envs/core/lib/python3.9/site-packages/cv2/../../lib64:
2023-01-09 09:12:43.281500: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-09 09:12:44.186134: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnv

In [2]:
labels = pd.read_csv('label_names.csv')
colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

In [3]:
def video(model_name, video_name, result_name):
    labels = pd.read_csv('label_names.csv')
    model = load_model(model_name)
    
    path_to_weights = 'yolov3_ts_train_5000.weights'
    path_to_cfg = 'yolov3_ts_test.cfg'

    # Cargando el modelo entrenado con YOLO v3
    network = cv2.dnn.readNetFromDarknet(path_to_cfg, path_to_weights)

    network.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
    network.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL_FP16)
    
    # Nombre de todas las capas YOLO v3
    layers_all = network.getLayerNames()


    # Imprimimos solo las capas de deteccion YOLO v3: 82, 94 y 106
    layers_names_output = [layers_all[i - 1] for i in network.getUnconnectedOutLayers()]

    # Check point
    #print()
    #print(layers_names_output)  # ['yolo_82', 'yolo_94', 'yolo_106']
    
    # Minima prob. para eliminar detecciones
    probability_minimum = 0.2

    #Setting threshold to filtering weak bounding boxes by non-maximum suppression
    threshold = 0.2
    
    #video = cv2.VideoCapture('traffic-sign-to-test.mp4')
    video = cv2.VideoCapture(video_name)
    # Writerque se usará para el procesado de frames
    writer = None

    # Variables para las dimenciones espaciales de los frames
    h, w = None, None


    %matplotlib inline

    # Setting default size of plots
    plt.rcParams['figure.figsize'] = (3, 3)

    # Variable for counting total amount of frames
    f = 0

    # Variable for counting total processing time
    t = 0

    # Catching frames in the loop
    while True:
        # Capturing frames one-by-one
        ret, frame = video.read()

        # If the frame was not retrieved
        if not ret:
            break
       
        # Getting spatial dimensions of the frame for the first time
        if w is None or h is None:
            # Slicing two elements from tuple
            h, w = frame.shape[:2]

        # Blob from current frame
        blob = cv2.dnn.blobFromImage(cv2.cvtColor(frame,  cv2.COLOR_BGR2RGB), 1 / 255.0, (416, 416), swapRB=True, crop=False)

        # Forward pass with blob through output layers
        network.setInput(blob)
        start = time.time()
        output_from_network = network.forward(layers_names_output)
        end = time.time()

        # Increasing counters
        f += 1
        t += end - start

        # Spent time for current frame
        #print('Frame number {0} took {1:.5f} seconds'.format(f, end - start))

        # Lists for detected bounding boxes, confidences and class's number
        bounding_boxes = []
        confidences = []
        #class_numbers = []

        # Going through all output layers after feed forward pass
        for result in output_from_network:
            # Going through all detections from current output layer
            for detected_objects in result:
                # Getting 80 classes' probabilities for current detected object
                scores = detected_objects[5:]
                # Getting index of the class with the maximum value of probability
                class_current = np.argmax(scores)
                # Getting value of probability for defined class
                confidence_current = scores[class_current]

                # Eliminating weak predictions by minimum probability
                if confidence_current > probability_minimum:
                    # Scaling bounding box coordinates to the initial frame size
                    box_current = detected_objects[0:4] * np.array([w, h, w, h])

                    # Getting top left corner coordinates
                    x_center, y_center, box_width, box_height = box_current
                    x_min = int(x_center - (box_width / 2))
                    y_min = int(y_center - (box_height / 2))

                    # Adding results into prepared lists
                    bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
                    confidences.append(float(confidence_current))
                    #class_numbers.append(class_current)
                

        # Implementing non-maximum suppression of given bounding boxes
        results = cv2.dnn.NMSBoxes(bounding_boxes, confidences, probability_minimum, threshold)

        # Checking if there is any detected object been left
        if len(results) > 0:
            # Going through indexes of results
            for i in results.flatten():
                # Bounding box coordinates, its width and height
                x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
                box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]
            
            
                # Cut fragment with Traffic Sign
                c_ts = frame[y_min:y_min+int(box_height), x_min:x_min+int(box_width), :]
                # print(c_ts.shape)
            
                if c_ts.shape[:1] == (0,) or c_ts.shape[1:2] == (0,):
                    pass
                else:
                    # Getting preprocessed blob with Traffic Sign of needed shape
                    blob_ts = cv2.dnn.blobFromImage(cv2.cvtColor(c_ts,  cv2.COLOR_BGR2RGB), 1 / 255.0, size=(32, 32), swapRB=True, crop=False)
                    #blob_ts[0] = blob_ts[0, :, :, :] - mean['mean_image_rgb']
                    blob_ts = blob_ts.transpose(0, 2, 3, 1)


                    # Feeding to the Keras CNN model to get predicted label among 43 classes
                    scores = model.predict(blob_ts)

                    # Scores is given for image with 43 numbers of predictions for each class
                    # Getting only one class with maximum value
                    prediction = np.argmax(scores)
                    # print(labels['SignName'][prediction])


                    # Colour for current bounding box
                    colour_box_current = colours[prediction].tolist()

                    # Drawing bounding box on the original current frame
                    cv2.rectangle(frame, (x_min, y_min),
                              (x_min + box_width, y_min + box_height),
                              colour_box_current, 2)

                    # Preparing text with label and confidence for current bounding box
                    text_box_current = '{}: {:.4f}'.format(labels['SignName'][prediction],
                                                       confidences[i])
                    # Putting text with label and confidence on the original image
                    cv2.putText(frame, text_box_current, (x_min, y_min - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, colour_box_current, 2)


        # Initializing writer only once
        if writer is None:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')

            # Writing current processed frame into the video file
            writer = cv2.VideoWriter(result_name, fourcc, 25,
                                 (frame.shape[1], frame.shape[0]), True)

        # Write processed current frame to the file
        writer.write(frame)


    # Releasing video reader and writer
    video.release()
    writer.release()    
    
    print('Total number of frames', f)
    #print('Total amount of time {:.5f} seconds'.format(t))
    print('FPS:', round((f / t), 1))

    # Saving locally without committing


    FileLink(result_name)



In [4]:
path = os.path.join(os.getcwd(),'videos_prueba')
path_resul = os.path.join(os.getcwd(),'resultados')
for video_name in os.listdir(path):
    print(video_name)
    video('../datos/modelos_finales/model_cnn.h5',
          os.path.join(path, video_name),
          os.path.join(path_resul, "result_"+video_name[:-4]+".mp4"))

IMG_9560.MOV


2023-01-08 20:35:53.862511: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/teresababio/miniconda3/envs/core/lib/python3.9/site-packages/cv2/../../lib64:
2023-01-08 20:35:53.862689: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-08 20:35:53.862781: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-FNGN17J8): /proc/driver/nvidia/version does not exist
2023-01-08 20:35:53.864367: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow wit


['yolo_82', 'yolo_94', 'yolo_106']


[ WARN:0@6.890] global /io/opencv/modules/dnn/src/net_impl.cpp (154) setUpNet DNN: OpenCL target is not supported with current OpenCL device (tested with GPUs only), switching to CPU.


1/1 [==============================] - 0s 26ms/step


...

1/1 [==============================] - 0s 37ms/step
Total number of frames 231
FPS: 0.8
IMG_9552.MOV

['yolo_82', 'yolo_94', 'yolo_106']


[ WARN:0@11152.885] global /io/opencv/modules/dnn/src/net_impl.cpp (154) setUpNet DNN: OpenCL target is not supported with current OpenCL device (tested with GPUs only), switching to CPU.


1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 26ms/step


1/1 [==============================] - 0s 28ms/step
Total number of frames 716
FPS: 0.8
